In [13]:
import os
import pydicom as dcm
from tools import *

In [14]:
PATH = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/'

## PTV Cutoff

In [15]:
list_all_patients = sorted([p for p in os.listdir(PATH) if 'b' not in p and 'old' not in p], key=int,reverse=False)

In [119]:
PTV_cutoffs = {}
no_eyes = []
for p in list_all_patients:
    print("******", p , "*******")
    patient_path = PATH+p+'/'
    CT_list = [d for d in os.listdir(patient_path) if d[9:11] == 'CT' and len(d) == 23]
    if len(CT_list) > 2:
        print(p,"has",len(CT_list),"CTs.")
    PTV_cutoffs[p] = {}    
    for CT in CT_list:
        CT_path = patient_path+CT+'/'
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        
#         print(p, CT, RS_file)
        
        
        eye_ROI_names = find_ROI_names(RS,keyword='eye')
        
        if len(eye_ROI_names) != 2:
            print(p, eye_ROI_names)
        if len(eye_ROI_names) == 0:
            
            eye_ROI_names = find_ROI_names(RS,keyword='orbit')
            if len(eye_ROI_names) == 0:
                 eye_ROI_names = find_ROI_names(RS,keyword='globe')
            if len(eye_ROI_names) == 0:
                print("***Patient", p, "CT",CT, "has no eye contours... ***")
                no_eyes.append(p)
                print(find_ROI_names(RS))
                continue
   
            
        dict_contours, z_lists = get_all_ROI_contours(eye_ROI_names, RS)
#         all_z_slices = sorted(z_lists[0] + [z for z in z_lists[1] if z not in z_lists[0]])
        all_z_slices = []
        for z_list in z_lists:
            all_z_slices = sorted(all_z_slices + [z for z in z_list if z not in all_z_slices])
        
        PTV_names = [p for p in find_ROI_names(RS,"PTV") if '!' not in p and 'z' not in p and 'E-PTV' not in p and 'NON' not in []]
#         print(PTV_names)
        PTV_dict_contours, z_lists = get_all_ROI_contours(PTV_names, RS)
        
        all_z_slices_PTV = []
        for z_list in z_lists:
            all_z_slices_PTV = sorted(all_z_slices_PTV + [z for z in z_list if z not in all_z_slices_PTV])
        
        PTV_slices_cut = 0
        len_PTV_slices = len(all_z_slices_PTV)
        
        for z in all_z_slices_PTV:
            if z in all_z_slices or z>min(all_z_slices):
                PTV_slices_cut += 1
        
        
        PTV_cutoffs[p][CT] = {}
        PTV_cutoffs[p][CT]['total'] = len_PTV_slices
        PTV_cutoffs[p][CT]['cut'] = PTV_slices_cut
        
        

        

****** 19 *******
****** 22 *******
****** 23 *******
****** 35 *******
****** 43 *******
****** 51 *******
****** 54 *******
****** 59 *******
****** 60 *******
****** 70 *******
****** 77 *******
****** 81 *******
****** 100 *******
****** 101 *******
****** 103 *******
****** 108 *******
****** 112 *******
****** 115 *******
****** 118 *******
****** 146 *******
****** 147 *******
****** 153 *******
****** 157 *******
****** 169 *******
****** 172 *******
****** 248 *******
****** 257 *******
****** 272 *******
****** 274 *******
****** 294 *******
****** 359 *******
****** 384 *******
****** 390 *******
****** 408 *******
****** 424 *******
****** 425 *******
****** 432 *******
****** 436 *******
****** 437 *******
****** 439 *******
****** 440 *******
****** 443 *******
****** 447 *******
****** 448 *******
****** 449 *******
****** 451 *******
****** 452 *******
****** 453 *******
****** 454 *******
****** 456 *******
****** 458 *******
458 ['Eye_R']
****** 459 *******
****** 460

In [120]:
# Number of patients counted
total_ps = 0
for p in PTV_cutoffs:
    if bool(PTV_cutoffs[p]) and p not in remove:# and p!='1514':
        
        total_ps+=1

total_CTs = 0
for p in PTV_cutoffs:
    for CT in PTV_cutoffs[p]:
        if p not in remove:
            total_CTs+=1
        
print("Total patients with dirs:",len(list_all_patients)-len(remove))
print("Total patients counted:", total_ps)
print("Total CTs counted:",total_CTs)
print("Patients without eyes:",len(no_eyes))

Total patients with dirs: 656
Total patients counted: 609
Total CTs counted: 828
Patients without eyes: 37


In [121]:
for p in no_eyes:
    CT_list = [d for d in os.listdir(PATH+p) if d[9:11] == 'CT' and len(d) == 23]
    for CT in CT_list:
        print("*****",p,"***")
        CT_path = PATH+p+'/'+CT+'/'
        print(CT)
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        print("EYE:",find_ROI_names(RS,keyword='eye'))
        print("ORBIT:",find_ROI_names(RS,keyword='orbit'))
        print("GLOBE:",find_ROI_names(RS,keyword='globe'))

***** 853 ***
20200304_CT_26_FEB_2020
EYE: []
ORBIT: []
GLOBE: []
***** 1467 ***
20230920_CT_20_SEP_2023
EYE: []
ORBIT: []
GLOBE: []
***** 1467 ***
20230907_CT_01_SEP_2023
EYE: ['Eye_R', 'Eye_L']
ORBIT: []
GLOBE: []
***** 1611 ***
20210309_CT_30_JUN_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1641 ***
20210308_CT_16_OCT_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1645 ***
20210226_CT_02_NOV_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1651 ***
20210226_CT_11_DEC_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1655 ***
20210226_CT_03_JAN_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1662 ***
20210308_CT_01_MAR_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1694 ***
20210309_CT_25_SEP_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1695 ***
20210226_CT_04_OCT_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1699 ***
20210309_CT_05_NOV_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1700 ***
20181116_CT_08_NOV_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1723 ***
20190627_CT_25_APR_2019
EYE: []
ORBIT: []
GLOBE: []
***** 1723 ***
20190628_CT_18_JUN_2019
EYE: ['E

In [122]:

CT_list = [d for d in os.listdir(PATH+p) if d[9:11] == 'CT' and len(d) == 23]
for CT in CT_list:
    print("*****",p,"***")
    CT_path = PATH+p+'/'+CT+'/'
    RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]

***** 2075 ***


In [123]:
PTV_cutoffs

{'19': {'20180411_CT_09_APR_2018': {'total': 92, 'cut': 0},
  '20180515_CT_10_MAY_2018': {'total': 92, 'cut': 0}},
 '22': {'20180921_CT_13_SEP_2018': {'total': 67, 'cut': 0}},
 '23': {'20180831_CT_22_AUG_2018': {'total': 96, 'cut': 0}},
 '35': {'20180209_CT_06_FEB_2018': {'total': 94, 'cut': 0},
  '20171220_CT_14_DEC_2017': {'total': 93, 'cut': 0}},
 '43': {'20190131_CT_24_JAN_2019': {'total': 71, 'cut': 0}},
 '51': {'20190430_CT_24_APR_2019': {'total': 77, 'cut': 0}},
 '54': {'20190122_CT_14_JAN_2019': {'total': 49, 'cut': 11},
  '20190225_CT_19_FEB_2019': {'total': 40, 'cut': 13}},
 '59': {'20180807_CT_31_JUL_2018': {'total': 100, 'cut': 4},
  '20180705_CT_27_JUN_2018': {'total': 140, 'cut': 5}},
 '60': {'20181004_CT_26_SEP_2018': {'total': 84, 'cut': 0}},
 '70': {'20180221_CT_14_FEB_2018': {'total': 88, 'cut': 0},
  '20180403_CT_28_MAR_2018': {'total': 120, 'cut': 0}},
 '77': {'20190320_CT_13_MAR_2019': {'total': 29, 'cut': 0}},
 '81': {'20190125_CT_16_JAN_2019': {'total': 97, 'cut'

In [124]:
CT_percentages = []

num_CTs = 0
for patient in PTV_cutoffs:
    for CT in PTV_cutoffs[patient]:
        if PTV_cutoffs[patient][CT]['total'] == 0:
            print("***",patient,PTV_cutoffs[patient],"***")
        else:
            num_CTs += 1
            percentage = PTV_cutoffs[patient][CT]['cut']/PTV_cutoffs[patient][CT]['total']*100
            CT_percentages.append(percentage)
            if percentage > 25:
                print(patient,":", percentage,"% -->",PTV_cutoffs[patient][CT]['cut'],"/",PTV_cutoffs[patient][CT]['total'])
    

54 : 32.5 % --> 13 / 40
108 : 54.54545454545454 % --> 18 / 33
248 : 44.61538461538462 % --> 29 / 65
466 : 38.46153846153847 % --> 10 / 26
638 : 32.432432432432435 % --> 12 / 37
638 : 42.857142857142854 % --> 9 / 21
*** 641 {'20191022_CT_15_OCT_2019': {'total': 0, 'cut': 0}} ***
690 : 37.719298245614034 % --> 43 / 114
868 : 40.42553191489361 % --> 19 / 47
936 : 51.724137931034484 % --> 15 / 29
980 : 50.0 % --> 10 / 20
1071 : 42.857142857142854 % --> 24 / 56
1130 : 38.333333333333336 % --> 23 / 60
1143 : 61.42857142857143 % --> 43 / 70
1306 : 26.5625 % --> 34 / 128
1306 : 29.496402877697843 % --> 82 / 278
1467 : 45.45454545454545 % --> 15 / 33
1489 : 45.83333333333333 % --> 11 / 24
1514 : 44.44444444444444 % --> 8 / 18
1521 : 58.82352941176471 % --> 10 / 17
1607 : 51.68539325842697 % --> 46 / 89
1659 : 25.806451612903224 % --> 40 / 155
1686 : 31.428571428571427 % --> 11 / 35
1923 : 28.915662650602407 % --> 24 / 83
1951 : 45.45454545454545 % --> 10 / 22
1951 : 61.904761904761905 % --> 13 

In [105]:
CT_slices = []
PTVs_below_cut = 0
for patient in PTV_cutoffs:
    for CT in PTV_cutoffs[patient]:
        CT_slices.append(PTV_cutoffs[patient][CT]['cut'])
        if PTV_cutoffs[patient][CT]['cut'] == 0:
            PTVs_below_cut+=1
    

In [106]:
def check_if_cut(slices, y_cutoff_roi, all_z_slices):
    if len(slices) == 0:
        return False
    for c_slice in slices:
        xo = []
        yo = []

        z = c_slice[2]
        if z in all_z_slices or z>min(all_z_slices):

#             xo = []
#             yo = []

            xi, yi, zi = c_slice[::3], c_slice[1::3], c_slice[2::3]
            for k,y_i in enumerate(yi):
                if y_i < y_cutoff_roi:
                    xo.append(xi[k])
                    yo.append(yi[k])

    if len(xo)==0 and len(yo)==0:
        return False
    return True

In [107]:
PTV_cut= {}

for p in sorted(list_all_patients,reverse=True,key=int):
    patient_path = PATH+p+'/'
    CT_list = [d for d in os.listdir(patient_path) if d[9:11] == 'CT' and len(d) == 23]
    if len(CT_list) > 2:
        print(p,"has",len(CT_list),"CTs.")
    PTV_cutoffs[p] = {}    
    for CT in CT_list:
        CT_path = patient_path+CT+'/'
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        
#         print(p, CT, RS_file)

        eye_ROI_names = [e for e in find_ROI_names(RS,keyword='eye') if '~' not in e]
#         print(eye_ROI_names)

        if len(eye_ROI_names) != 2:
            print(p, eye_ROI_names)
        if len(eye_ROI_names) == 0:
            print("***Patient", p, "CT",CT, "has no eye contours... ***")
            continue
            


        dict_contours, z_lists = get_all_ROI_contours(eye_ROI_names, RS)
#         print(z_lists)
        #         all_z_slices = sorted(z_lists[0] + [z for z in z_lists[1] if z not in z_lists[0]])
        all_z_slices = []
        for z_list in z_lists:
            all_z_slices = sorted(all_z_slices + [z for z in z_list if z not in all_z_slices])

        roi_slice, z_smg = get_avg_ROI_z_and_slice(z_lists)
#         print(roi_slice,z_smg)
        y_eye = []
        for eye in eye_ROI_names:
#             print(eye)
            roi_eye = dict_contours[eye][roi_slice] 
            for i in range(0,len(roi_eye),3):
                y_eye.append(roi_eye[i+1])
        max_eye = max(y_eye)
        min_eye = min(y_eye)
        y_cutoff_roi = ((max_eye-min_eye)/2+min_eye)
#         print(y_cutoff_roi)

        all_contour_names = find_ROI_names(RS)
        cut = []
        for name in all_contour_names:
            dict_cont, z_list = get_all_ROI_contours([name], RS)
            is_cut = check_if_cut(dict_cont[name], y_cutoff_roi, all_z_slices)
            if is_cut and 'lens' not in name.lower() and 'eye' not in name.lower():
                cut.append(name)
            if is_cut and 'ptv' in name.lower():
                if p not in PTV_cut:
                    PTV_cut[p] = {}
                if CT not in PTV_cut[p]:
                    PTV_cut[p][CT] = []
                
                PTV_cut[p][CT].append(name)
                

        print(p, cut)
        # dict_contours_all, z_lists_all = get_all_ROI_contours(all_contour_names, RS)

2078 []
2076 ['Skin']
2075 []
***Patient 2075 CT 20240306_CT_04_MAR_2024 has no eye contours... ***
2074 []
2073 []
2068 ['Skin']
2066 []
2062 []
2061 []
2061 []
2059 []
2058 []
2058 []
2057 []
2056 []
2056 []
2055 []
2054 ['Skin']
2051 ['Skin']
2047 ['Eye_R']
2047 []
2045 []
2044 []
2043 []
2043 ['Skin']
2042 []
2041 []
2041 []
2040 []
2037 []
2037 []
2036 []
2035 []
2035 ['CTV_6600']
2033 ['Skin']
2031 []
2031 []
2030 []
2027 []
2027 []
2026 []
2022 []
2022 []
2021 []
2020 []
***Patient 2020 CT 20231004_CT_21_SEP_2023 has no eye contours... ***
2019 []
2019 []
2018 []
2018 []
2017 []
2017 []
2015 []
2015 []
2013 ['Skin']
2013 ['Skin']
2003 []
2000 []
2000 []
1999 []
1999 []
1995 []
1994 []
1991 []
1987 has 3 CTs.
1987 ['Skin']
1987 ['Skin']
1987 ['Skin']
1982 []
1982 []
1978 []
1974 []
1969 []
1965 []
***Patient 1965 CT 20221229_CT_22_DEC_2022 has no eye contours... ***
1961 []
1960 []
1960 []
1955 []
1955 []
1954 ['Skin']
1951 ['Glnd_Lacrimal_L', 'Glnd_Lacrimal_R', 'Retina_R', 'Reti

In [116]:
count_CTs_PTV_cut = 0
for p in PTV_cut:
    for CT in PTV_cut[p]:
        if p not in remove:
            # print(p,CT)
            count_CTs_PTV_cut +=1

In [125]:
print("PTVs cropped:",count_CTs_PTV_cut,"/",total_CTs,"CTs =",count_CTs_PTV_cut/total_CTs*100,"%")

PTVs cropped: 42 / 828 CTs = 5.072463768115942 %


In [126]:
print("PTVs entirely below crop:",PTVs_below_cut,"/",total_CTs,"CTs =",PTVs_below_cut/total_CTs*100,"%")

PTVs entirely below crop: 708 / 828 CTs = 85.5072463768116 %


In [49]:
remove = ['1686','1434','980']

In [ ]:
1951 - C30.0
1923  - C31.0
1923 - C31.0
1901 - C30.0
1888 - C06.0
1888 - C06.0
1724 - C31.9
1723 - C11.9
1714 - C31.0
1686 20180823_CT_02_AUG_2018 -- remove from data
1607 C31.1
1598 C32.9
1551 C30.0
1521 C30.0
1514 C30.0
1489 C30.0
1488 C31.0
1467 C30.0
1438 C30.0
1434 remove
1432 C30.0
1402 C30.0
1209 C31.0
1143 C30.0
1130 C31.0
1071 C30.0
980 REMOVE
936 C30.0 CHECK WHY GONE in new cohort
915 C30.0
877 C30.0
877 C30.0
868 C30.0
863 C30.0
800 C05.0
798 C05.9
730 C30.0
690 C31.1
681 C05.9
638 C31.0
638 C30.0
466 C30.0
437 C30.0
437 C30.0
248 C30.0
108 C30.0

In [54]:
below_oral_cavity =    []

In [59]:
no_eyes

['853',
 '1467',
 '1611',
 '1641',
 '1645',
 '1651',
 '1655',
 '1662',
 '1694',
 '1695',
 '1699',
 '1700',
 '1723',
 '1726',
 '1736',
 '1761',
 '1780',
 '1787',
 '1790',
 '1809',
 '1821',
 '1822',
 '1824',
 '1832',
 '1850',
 '1860',
 '1865',
 '1873',
 '1880',
 '1886',
 '1889',
 '1899',
 '1907',
 '1940',
 '1965',
 '2020',
 '2075']

In [69]:
PTV_cutoffs_no_eyes = {}
no_oral = []
for p in  list(dict.fromkeys(no_eyes)):
    print("******", p , "*******")
    patient_path = PATH+p+'/'
    CT_list = [d for d in os.listdir(patient_path) if d[9:11] == 'CT' and len(d) == 23]
    if len(CT_list) > 2:
        print(p,"has",len(CT_list),"CTs.")
    PTV_cutoffs_no_eyes[p] = {}    
    for CT in CT_list:
        CT_path = patient_path+CT+'/'
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        
#         print(p, CT, RS_file)
        
        
        eye_ROI_names = find_ROI_names(RS,keyword='chiasm')
        
        if len(eye_ROI_names) != 2:
            print(p, eye_ROI_names)
        if len(eye_ROI_names) == 0:
            
            eye_ROI_names = find_ROI_names(RS,keyword='brain')
            if len(eye_ROI_names) == 0:
                 eye_ROI_names = find_ROI_names(RS,keyword='globe')
            if len(eye_ROI_names) == 0:
                print("***Patient", p, "CT",CT, "has no eye contours... ***")
                no_oral.append(p)
                print(find_ROI_names(RS))
                continue
   
            
        dict_contours, z_lists = get_all_ROI_contours(eye_ROI_names, RS)
#         all_z_slices = sorted(z_lists[0] + [z for z in z_lists[1] if z not in z_lists[0]])
        all_z_slices = []
        for z_list in z_lists:
            all_z_slices = sorted(all_z_slices + [z for z in z_list if z not in all_z_slices])
        
        PTV_names = [p for p in find_ROI_names(RS,"PTV") if '!' not in p and 'z' not in p and 'E-PTV' not in p and 'NON' not in []]
#         print(PTV_names)
        PTV_dict_contours, z_lists = get_all_ROI_contours(PTV_names, RS)
        
        all_z_slices_PTV = []
        for z_list in z_lists:
            all_z_slices_PTV = sorted(all_z_slices_PTV + [z for z in z_list if z not in all_z_slices_PTV])
        
        PTV_slices_cut = 0
        len_PTV_slices = len(all_z_slices_PTV)
        
        for z in all_z_slices_PTV:
            if z in all_z_slices or z>min(all_z_slices):
                PTV_slices_cut += 1
        
        
        PTV_cutoffs_no_eyes[p][CT] = {}
        PTV_cutoffs_no_eyes[p][CT]['total'] = len_PTV_slices
        PTV_cutoffs_no_eyes[p][CT]['cut'] = PTV_slices_cut
        
        

        

****** 853 *******
853 []
****** 1467 *******
1467 []
***Patient 1467 CT 20230920_CT_20_SEP_2023 has no eye contours... ***
['BODY']
****** 1611 *******
1611 []
****** 1641 *******
1641 []
****** 1645 *******
1645 []
****** 1651 *******
1651 []
****** 1655 *******
1655 []
****** 1662 *******
1662 []
****** 1694 *******
1694 []
****** 1695 *******
1695 []
****** 1699 *******
1699 ['Chiasm']
****** 1700 *******
1700 []
****** 1723 *******
1723 ['OpticChiasm']
****** 1726 *******
1726 []
****** 1736 *******
1736 []
****** 1761 *******
1761 ['OpticChiasm']
****** 1780 *******
1780 []
****** 1787 *******
1787 []
****** 1790 *******
1790 []
****** 1809 *******
1809 []
****** 1821 *******
1821 []
****** 1822 *******
1822 []
****** 1824 *******
1824 []
****** 1832 *******
1832 []
****** 1850 *******
1850 []
****** 1860 *******
1860 []
****** 1865 *******
1865 []
****** 1873 *******
1873 []
****** 1880 *******
1880 []
****** 1886 *******
1886 []
****** 1889 *******
1889 []
****** 1899 *******
1

In [90]:
# Number of patients counted
total_ps = 0
for p in PTV_cutoffs_no_eyes:
    if bool(PTV_cutoffs_no_eyes[p]):# and p!='1514':
        total_ps+=1

total_CTs_noeyes = 0
for p in PTV_cutoffs_no_eyes:
    for CT in PTV_cutoffs_no_eyes[p]:
        total_CTs_noeyes+=1
        
print("Total patients with dirs:",len(list_all_patients))
print("Total patients counted:", total_ps)
print("Total CTs counted:",total_CTs_noeyes)
print("Patients without eyes:",len(no_oral))

Total patients with dirs: 659
Total patients counted: 37
Total CTs counted: 38
Patients without eyes: 1


In [32]:
for p in no_eyes:
    CT_list = [d for d in os.listdir(PATH+p) if d[9:11] == 'CT' and len(d) == 23]
    for CT in CT_list:
        print("*****",p,"***")
        CT_path = PATH+p+'/'+CT+'/'
        print(CT)
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        print("EYE:",find_ROI_names(RS,keyword='eye'))
        print("ORBIT:",find_ROI_names(RS,keyword='orbit'))
        print("GLOBE:",find_ROI_names(RS,keyword='globe'))

***** 853 ***
20200304_CT_26_FEB_2020
EYE: []
ORBIT: []
GLOBE: []
***** 1467 ***
20230920_CT_20_SEP_2023
EYE: []
ORBIT: []
GLOBE: []
***** 1467 ***
20230907_CT_01_SEP_2023
EYE: ['Eye_R', 'Eye_L']
ORBIT: []
GLOBE: []
***** 1611 ***
20210309_CT_30_JUN_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1641 ***
20210308_CT_16_OCT_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1645 ***
20210226_CT_02_NOV_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1651 ***
20210226_CT_11_DEC_2017
EYE: []
ORBIT: []
GLOBE: []
***** 1655 ***
20210226_CT_03_JAN_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1662 ***
20210308_CT_01_MAR_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1694 ***
20210309_CT_25_SEP_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1695 ***
20210226_CT_04_OCT_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1699 ***
20210309_CT_05_NOV_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1700 ***
20181116_CT_08_NOV_2018
EYE: []
ORBIT: []
GLOBE: []
***** 1723 ***
20190627_CT_25_APR_2019
EYE: []
ORBIT: []
GLOBE: []
***** 1723 ***
20190628_CT_18_JUN_2019
EYE: ['E

In [ ]:

CT_list = [d for d in os.listdir(PATH+p) if d[9:11] == 'CT' and len(d) == 23]
for CT in CT_list:
    print("*****",p,"***")
    CT_path = PATH+p+'/'+CT+'/'
    RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]

In [33]:
manually checking = {
    '2075'
}

SyntaxError: invalid syntax (2941610387.py, line 1)

In [35]:
PTVs_cut

NameError: name 'PTVs_cut' is not defined

In [64]:
PTV_cutoffs

{'19': {'20180411_CT_09_APR_2018': {'total': 92, 'cut': 0},
  '20180515_CT_10_MAY_2018': {'total': 92, 'cut': 0}},
 '22': {'20180921_CT_13_SEP_2018': {'total': 67, 'cut': 0}},
 '23': {'20180831_CT_22_AUG_2018': {'total': 96, 'cut': 0}},
 '35': {'20180209_CT_06_FEB_2018': {'total': 94, 'cut': 0},
  '20171220_CT_14_DEC_2017': {'total': 93, 'cut': 0}},
 '43': {'20190131_CT_24_JAN_2019': {'total': 71, 'cut': 0}},
 '51': {'20190430_CT_24_APR_2019': {'total': 77, 'cut': 0}},
 '54': {'20190122_CT_14_JAN_2019': {'total': 49, 'cut': 11},
  '20190225_CT_19_FEB_2019': {'total': 40, 'cut': 13}},
 '59': {'20180807_CT_31_JUL_2018': {'total': 100, 'cut': 4},
  '20180705_CT_27_JUN_2018': {'total': 140, 'cut': 5}},
 '60': {'20181004_CT_26_SEP_2018': {'total': 84, 'cut': 0}},
 '70': {'20180221_CT_14_FEB_2018': {'total': 88, 'cut': 0},
  '20180403_CT_28_MAR_2018': {'total': 120, 'cut': 0}},
 '77': {'20190320_CT_13_MAR_2019': {'total': 29, 'cut': 0}},
 '81': {'20190125_CT_16_JAN_2019': {'total': 97, 'cut'

In [71]:
CT_percentages = []

num_CTs = 0
for patient in PTV_cutoffs_no_eyes:
    for CT in PTV_cutoffs_no_eyes[patient]:
        if PTV_cutoffs_no_eyes[patient][CT]['total'] == 0:
            print("***",patient,PTV_cutoffs[patient],"***")
        else:
            num_CTs += 1
            percentage = PTV_cutoffs_no_eyes[patient][CT]['cut']/PTV_cutoffs_no_eyes[patient][CT]['total']*100
            CT_percentages.append(percentage)
            if percentage > 25:
                print(patient,":", percentage,"% -->",PTV_cutoffs_no_eyes[patient][CT]['cut'],"/",PTV_cutoffs_no_eyes[patient][CT]['total'])
    

1467 : 60.60606060606061 % --> 20 / 33
1790 : 32.432432432432435 % --> 12 / 37


In [87]:
CT_slices = []
PTVs_below_cut_noeyes = 0
for patient in PTV_cutoffs_no_eyes:
    for CT in PTV_cutoffs_no_eyes[patient]:
        CT_slices.append(PTV_cutoffs_no_eyes[patient][CT]['cut'])
        if PTV_cutoffs_no_eyes[patient][CT]['cut'] == 0:
            PTVs_below_cut_noeyes+=1
            print("G",patient)
        else:
            print(patient)
    

853
1467
G 1611
G 1641
G 1645
G 1651
G 1655
G 1662
G 1694
G 1695
G 1699
G 1700
G 1723
1723
G 1726
G 1736
G 1761
1780
1787
1790
G 1809
G 1821
G 1822
G 1824
G 1832
G 1850
G 1860
1865
G 1873
G 1880
G 1886
1889
G 1899
G 1907
G 1940
G 1965
G 2020
2075


In [74]:
def check_if_cut(slices, y_cutoff_roi, all_z_slices):
    if len(slices) == 0:
        return False
    for c_slice in slices:
        xo = []
        yo = []

        z = c_slice[2]
        if z in all_z_slices or z>min(all_z_slices):

#             xo = []
#             yo = []

            xi, yi, zi = c_slice[::3], c_slice[1::3], c_slice[2::3]
            for k,y_i in enumerate(yi):
                if y_i < y_cutoff_roi:
                    xo.append(xi[k])
                    yo.append(yi[k])

    if len(xo)==0 and len(yo)==0:
        return False
    return True

In [77]:
PTV_cut_no_eyes= {}

for p in sorted( list(dict.fromkeys(no_eyes)),reverse=True,key=int):
    patient_path = PATH+p+'/'
    CT_list = [d for d in os.listdir(patient_path) if d[9:11] == 'CT' and len(d) == 23]
    if len(CT_list) > 2:
        print(p,"has",len(CT_list),"CTs.")
    PTV_cutoffs[p] = {}    
    for CT in CT_list:
        CT_path = patient_path+CT+'/'
        RS_file = [rs for rs in os.listdir(CT_path) if 'RS' in rs][0]
        RS = dcm.read_file(CT_path+RS_file)
        
#         print(p, CT, RS_file)

        eye_ROI_names = [e for e in find_ROI_names(RS,keyword='chiasma') if '~' not in e]
#         print(eye_ROI_names)
        

        if len(eye_ROI_names) != 2:
            print(p, eye_ROI_names)
        if len(eye_ROI_names) == 0:
            eye_ROI_names = [e for e in find_ROI_names(RS,keyword='brain') if '~' not in e]
            print("***Patient", p, "CT",CT, "has no eye contours... ***")
            continue
            


        dict_contours, z_lists = get_all_ROI_contours(eye_ROI_names, RS)
#         print(z_lists)
        #         all_z_slices = sorted(z_lists[0] + [z for z in z_lists[1] if z not in z_lists[0]])
        all_z_slices = []
        for z_list in z_lists:
            all_z_slices = sorted(all_z_slices + [z for z in z_list if z not in all_z_slices])

        roi_slice, z_smg = get_avg_ROI_z_and_slice(z_lists)
#         print(roi_slice,z_smg)
        y_eye = []
        for eye in eye_ROI_names:
#             print(eye)
            roi_eye = dict_contours[eye][roi_slice] 
            for i in range(0,len(roi_eye),3):
                y_eye.append(roi_eye[i+1])
        max_eye = max(y_eye)
        min_eye = min(y_eye)
        y_cutoff_roi = ((max_eye-min_eye)/2+min_eye)
#         print(y_cutoff_roi)

        all_contour_names = find_ROI_names(RS)
        cut = []
        for name in all_contour_names:
            dict_cont, z_list = get_all_ROI_contours([name], RS)
            is_cut = check_if_cut(dict_cont[name], y_cutoff_roi, all_z_slices)
            if is_cut and 'lens' not in name.lower() and 'eye' not in name.lower():
                cut.append(name)
            if is_cut and 'ptv' in name.lower():
                if p not in PTV_cut:
                    PTV_cut_no_eyes[p] = {}
                if CT not in PTV_cut[p]:
                    PTV_cut_no_eyes[p][CT] = []
                
                PTV_cut_no_eyes[p][CT].append(name)
                

        print(p, cut)
        # dict_contours_all, z_lists_all = get_all_ROI_contours(all_contour_names, RS)

2075 []
***Patient 2075 CT 20240306_CT_04_MAR_2024 has no eye contours... ***
2020 []
***Patient 2020 CT 20231004_CT_21_SEP_2023 has no eye contours... ***
1965 []
***Patient 1965 CT 20221229_CT_22_DEC_2022 has no eye contours... ***
1940 []
***Patient 1940 CT 20220818_CT_15_AUG_2022 has no eye contours... ***
1907 []
***Patient 1907 CT 20220224_CT_17_FEB_2022 has no eye contours... ***
1899 []
***Patient 1899 CT 20220127_CT_24_JAN_2022 has no eye contours... ***
1889 []
***Patient 1889 CT 20211117_CT_10_NOV_2021 has no eye contours... ***
1886 []
***Patient 1886 CT 20211027_CT_26_OCT_2021 has no eye contours... ***
1880 []
***Patient 1880 CT 20210922_CT_21_SEP_2021 has no eye contours... ***
1873 []
***Patient 1873 CT 20210816_CT_12_AUG_2021 has no eye contours... ***
1865 []
***Patient 1865 CT 20210629_CT_25_JUN_2021 has no eye contours... ***
1860 []
***Patient 1860 CT 20210526_CT_21_MAY_2021 has no eye contours... ***
1850 []
***Patient 1850 CT 20210414_CT_07_APR_2021 has no eye co

In [89]:
count_CTs_PTV_cut_noeyes = 0
for p in PTV_cut_no_eyes:
    for CT in PTV_cut_no_eyes[p]:
        if p not in remove:
            print(p,CT)
            count_CTs_PTV_cut_noeyes +=1

In [129]:
print("PTVs cropped:",count_CTs_PTV_cut_noeyes,"/",total_CTs_noeyes,"CTs =",count_CTs_PTV_cut_noeyes/total_CTs*100,"%")

PTVs cropped: 1 / 38 CTs = 0.12077294685990338 %


In [130]:
print("PTVs entirely below crop:",PTVs_below_cut_noeyes,"/",total_CTs_noeyes,"CTs =",PTVs_below_cut_noeyes/total_CTs*100,"%")

PTVs entirely below crop: 29 / 38 CTs = 3.5024154589371985 %


In [93]:
count_CTs_PTV_cut_noeyes+=1
PTVs_below_cut+=8

In [ ]:
853 -- all below
1467 -- croped
1723 -- lal below
1780 -- all below
1787 -- all below
1790 -- all below
1865 -- all beloe
1889 -- all belpw
2075 -- all below

In [127]:
print("PTVs entirely below crop:",PTVs_below_cut+PTVs_below_cut_noeyes,"/",total_CTs+total_CTs_noeyes,"CTs =",(PTVs_below_cut)/(total_CTs+total_CTs_noeyes)*100,"%")

PTVs entirely below crop: 737 / 866 CTs = 85.1039260969977 %


In [128]:
print("PTVs cropped:",count_CTs_PTV_cut_noeyes+count_CTs_PTV_cut,"/",total_CTs_noeyes+total_CTs,"CTs =",(count_CTs_PTV_cut+count_CTs_PTV_cut_noeyes)/(total_CTs+total_CTs_noeyes)*100,"%")

PTVs cropped: 43 / 866 CTs = 4.965357967667437 %


In [133]:
total = total_CTs+total_CTs_noeyes
overlap = total-PTVs_below_cut-PTVs_below_cut_noeyes-count_CTs_PTV_cut_noeyes-count_CTs_PTV_cut

In [134]:
print("PTVs within the cropped region:",overlap,"/",total,"CTs =",(overlap)/(total)*100,"%")

PTVs within the cropped region: 86 / 866 CTs = 9.930715935334874 %
